In [1]:
import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
url = "https://wrewsama.github.io/"
loader = WebBaseLoader(url)
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
splits = splitter.split_documents(docs)

[Document(page_content='wrewsamaYou need to enable JavaScript to run this app.', metadata={'source': 'https://wrewsama.github.io/', 'title': 'wrewsama', 'description': 'Web site created using create-react-app', 'language': 'en'})]

In [9]:
embeddings = OllamaEmbeddings(model='llama3')
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7bb1b1f250c0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [5]:
def infer(prompt):
    req = [{'role': 'user', 'content': prompt}]
    resp = ollama.chat(model='llama3', messages=req)
    return resp['message']['content']

In [6]:
retriever = vectorstore.as_retriever()
def infer_with_context(question):
    docs = retriever.invoke(question)
    ctx = '\n\n'.join(doc.page_content for doc in docs)
    prompt = f'Given this context: {ctx}\nAnswer this question: {question}'
    return infer(prompt)

NameError: name 'vectorstore' is not defined